# Using shallow water equations to understand large-scale dynamics
### MAQ - 32806, Chiel van Heerwaarden, 2016
In this tutorial you will use the previously derived shallow water equations to solve the famous Rossby adjustment problem in which a perturbation is added to a layer of fluid.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import animation
from IPython.display import HTML
figsize_s = 6,4
figsize_l = 10,4

In [ ]:
f_multiplier = 1.
lat = 52.
f = f_multiplier * 2.*7.29e-5*np.sin(np.pi/180.*lat)
g = 9.81
L = 6.371e6 * (2.*np.pi) * np.cos(np.pi/180.*lat)
L_dist = 1.e6 # Disturbance size.
h_ref = 200.
h_prime = 0.1

In [ ]:
nx = 1501
x = np.linspace(-L/2., L/2., nx)
dx = x[1]-x[0]
u0 = np.zeros(nx)
v0 = np.zeros(nx)

# Smooth step function.
# h0 = np.copy(x)
# h_sigma = 20.*dx
# for i in range(nx):
#    h0[i] = h_ref - h_prime*np.math.erf(h0[i]/(20*dx))

# Gaussian
x_sigma = L_dist/6.
h0 = np.exp(-x**2/(2.*x_sigma**2))
h0 = h_ref - h_prime*h0/np.max(h0)

# Mexican hat (Ricker wavelet).
# h_sigma = 25.*dx # The wavelet is approximately 8 h_sigma in total width.
# h_width = 2./((3.*h_sigma)**.5*np.pi**.25)
# h0 = 2./((3.*h_sigma)**.5*np.pi**.25) * (1. - x**2/h_sigma**2) * np.exp(-x**2/(2.*h_sigma**2))
# h0 = h_ref + h_prime*h0/np.max(h0)

# Just a check that the disturbance is well enough resolved.
pl.figure(figsize=figsize_s)
pl.plot(x/x_sigma, h0 - h_ref, 'bo-')
pl.xlim(-5, 5)
pl.xlabel('x / x_sigma')
pl.ylabel('h - h_ref');
pl.tight_layout()

In [ ]:
try:
    Lr = (g*h_ref)**.5 / f
except ZeroDivisionError:
    Lr = np.infty
finally:
    print("Circumference of Earth at latitude = {0:.1f} km".format(L/1.e3))
    print("Rossby radius of deformation = {0:.1f} km".format(Lr/1.e3))
    print("Size of disturbance = {0:.1f} km".format(L_dist/1.e3))
    print("Lr / L = {0}".format(Lr/L))
    print("Lr / x_sigma = {0}".format(Lr/x_sigma))

i_start, i_end = nx//2 - 6*(np.int(Lr/dx)+1), nx//2 + 6*(np.int(Lr/dx)+1)

In [ ]:
u = np.copy(u0)
v = np.copy(v0)
h = np.copy(h0)

nt = 50000
ni = 500
dt = 10.

u_out = np.zeros((nt//ni+1, nx))
v_out = np.zeros((nt//ni+1, nx))
h_out = np.zeros((nt//ni+1, nx))
u_out[0,:], v_out[0,:], h_out[0,:] = u0, v0, h0

for n in range(1,nt+1):
    u += dt * (  f*v - g*np.gradient(h,dx) )
    v += dt * ( -f*u )
    h += dt * ( -h_ref*np.gradient(u,dx) )
    
    if (n%ni == 0):
        u_out[n//ni,:] = u[:]
        v_out[n//ni,:] = v[:]
        h_out[n//ni,:] = h[:]

In [ ]:
pl.figure(figsize=figsize_l)
pl.subplot(121)
pl.plot(x/Lr, h , 'b-')
pl.plot(x/Lr, h0, 'b:')
pl.xlabel('x/Lr')
pl.ylabel('h')
pl.xlim(-8, 8)
pl.subplot(122)
pl.plot(x/Lr, u, 'b-', label='u')
pl.plot(x/Lr, v, 'r-', label='v')
pl.ylabel('u,v')
pl.xlim(-8, 8)
pl.legend(loc=0, frameon=False);
pl.tight_layout()

In [ ]:
q0 = f/h0
q  = (np.gradient(v,dx) + f) / h
pl.figure(figsize=figsize_s)
pl.plot(x/Lr, q , label='q_end  ')
pl.plot(x/Lr, q0, label='q_start')
pl.xlabel('x')
pl.ylabel('q')
pl.xlim(-8, 8)
pl.legend(loc=0, frameon=False);
pl.tight_layout()

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = pl.subplots(figsize=figsize_l)
xmin, xmax = min(x), max(x)
hmin, hmax = np.min(h_out), np.max(h_out)
hmin -= 0.05*(hmax-hmin)
hmax += 0.05*(hmax-hmin)
ax.set_xlim(xmin, xmax)
ax.set_ylim(hmin, hmax)
ax.set_xlabel('x')
ax.set_ylabel('h')
ax.plot(x, h_out[0,:], 'b:')
line, = ax.plot([], [], lw=2)
fig.tight_layout()

In [ ]:
# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(x, h_out[i,:])
    return (line,)

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=h_out.shape[0], interval=40, blit=True)

# call our new function to display the animation
HTML(anim.to_html5_video())

In [ ]:
print("Summing from x/Lr = ({0}, {1})".format(x[i_start]/Lr, x[i_end]/Lr))
print("A fraction {0} of the potential energy is retained near the disturbance."\
    .format(sum((h_out[-1,i_start:i_end]-h_ref)**2)/sum((h_out[0,i_start:i_end]-h_ref)**2)))
print("A fraction {0} of the initial height is retained."\
    .format( (h_out[-1,nx//2]-h_ref) / (h_out[0,nx//2]-h_ref) ))
    
pl.plot(x[i_start:i_end]/Lr, h_out[-1,i_start:i_end]-h_ref, lw=2)
pl.plot(x[i_start:i_end]/Lr, h_out[ 0,i_start:i_end]-h_ref, 'b--')
pl.xlim(x[i_start]/Lr, x[i_end]/Lr)
pl.xlabel('x/Lr')
pl.ylabel('h-h_ref')
pl.tight_layout();

In the previous tutorial, you have derived the potential vorticity. As you have discovered that the material derivative is conserved, the final solution can be found from the initial conditions, without solving the time varying equations. This can be done by matrix inversion.

In the steady state, we have
$$
0 = fv - g \dfrac{\partial h}{\partial x}\\
0 = -fu\\
0 = -H \dfrac{\partial u}{\partial x}
$$
and we know that potential vorticity has not changed:
$$
q = q_0
$$
___
* Derive an equation that relates $h$ to the initial potential vorticity $q_0$. This is a differential equation, you do not have to solve it.
___

In [ ]:
class steady_state:
    def __init__(self, f):
        q0 = f/h0
        c1 = g/(f*dx**2)
    
        # Matrix solver
        M = np.zeros((nx, nx))
    
        for i in range(1,nx-1):
            M[i,i-1] = c1
            M[i,i  ] = -(2.*c1 + q0[i])
            M[i,i+1] = c1

        M[ 0, 0] = 1.
        M[-1,-1] = 1.

        h_mat = -1.*np.ones(nx)*f
        h_mat[ 0] = h_ref
        h_mat[-1] = h_ref

        self.h = np.linalg.inv(M).dot(h_mat)
        self.v = g/f*np.gradient(self.h, dx)
        
        self.Lr = (g*h_ref)**.5/f
        
sol = steady_state(f)

# Plot solutions.    
pl.figure(figsize=figsize_l)
pl.subplot(121)
pl.plot(x/Lr, h_out[ 0,:], 'k:', label='initial')
pl.plot(x/Lr, sol.h, 'y-', lw=2, label='analytical')
pl.plot(x/Lr, h_out[-1,:], 'k--', lw=2, label='simulation')
pl.xlim(-20, 20)
pl.xlabel('x/Lr')
pl.ylabel('h')
pl.legend(loc=0, frameon=False)
pl.subplot(122)
pl.plot(x/Lr, v_out[ 0,:], 'k:', label='initial')
pl.plot(x/Lr, sol.v, 'y-', lw=2, label='analytical')
pl.plot(x/Lr, v_out[-1,:], 'k--', lw=2, label='simulation')
pl.xlim(-20, 20)
pl.xlabel('x/Lr')
pl.ylabel('v')
pl.legend(loc=0, frameon=False)
pl.tight_layout()

In [ ]:
f_half = 0.5*f
f_double = 2.*f

sol_ref = steady_state(f)
sol_half = steady_state(f_half)
sol_double = steady_state(f_double)

pl.figure(figsize=figsize_l)
pl.subplot(121)
pl.plot(x, sol_ref.h    - h_ref, 'b-')
pl.plot(x, sol_half.h   - h_ref, 'r-')
pl.plot(x, sol_double.h - h_ref, 'g-')
pl.plot(x, h0 - h_ref, 'k:')
pl.xlabel('x')
pl.ylabel('h')
pl.xlim(-30*x_sigma, 30*x_sigma)
pl.subplot(122)
pl.plot(x, sol_ref.v   , 'b-', label='ref'   )
pl.plot(x, sol_half.v  , 'r-', label='half'  )
pl.plot(x, sol_double.v, 'g-', label='double')
pl.xlabel('x')
pl.ylabel('v')
pl.xlim(-30*x_sigma, 30*x_sigma)
pl.legend(loc=0, frameon=False);
pl.tight_layout()

In [ ]:
pl.figure(figsize=figsize_l)
pl.subplot(121)
pl.plot(x/sol_ref.Lr   , (sol_ref.h    - h_ref)/sol_ref.Lr   , 'b-')
pl.plot(x/sol_half.Lr  , (sol_half.h   - h_ref)/sol_half.Lr  , 'r-')
pl.plot(x/sol_double.Lr, (sol_double.h - h_ref)/sol_double.Lr, 'g-')
pl.xlabel('x/Lr')
pl.ylabel('h')
pl.xlim(-8, 8)
pl.subplot(122)
pl.plot(x/sol_ref.Lr   , sol_ref.v   , 'b-', label='ref'   )
pl.plot(x/sol_half.Lr  , sol_half.v  , 'r-', label='half'  )
pl.plot(x/sol_double.Lr, sol_double.v, 'g-', label='double')
pl.xlabel('x/Lr')
pl.ylabel('v')
pl.xlim(-8, 8)
pl.legend(loc=0, frameon=False);
pl.tight_layout()

___
End of tutorial.
___